## Poker Hand

In this challenge, we have to find out which kind of Poker combination is present in a deck of 5 cards.Every card is a string containing the card value (with the upper-case initial for face-cards) and the lower-case initial for suits, as in the examples below:

> "Ah" ➞ Ace of hearts <br>
> "Ks" ➞ King of spades<br>
> "3d" ➞ Three of diamonds<br>
> "Qc" ➞ Queen of clubs <br>

There are 10 different combinations. Here's the list, in decreasing order of importance:

| Name            | Description                                         |
|-----------------|-----------------------------------------------------|
| Royal Flush     | A, K, Q, J, 10, all with the same suit.             |
| Straight Flush  | Five cards in sequence, all with the same suit.     |
| Four of a Kind  | Four cards of the same rank.                        |
| Full House      | Three of a Kind with a Pair.                        |
| Flush           | Any five cards of the same suit, not in sequence    |
| Straight        | Five cards in a sequence, but not of the same suit. |
| Three of a Kind | Three cards of the same rank.                       |
| Two Pair        | Two different Pairs.                                |
| Pair            | Two cards of the same rank.                         |
| High Card       | No other valid combination.                         |

### 1. Given a list `hand` containing five strings being the cards, implement a function `poker_hand_ranking` that returns a string with the name of the **highest** combination obtained, accordingly to the table above.

#### Examples

> poker_hand_ranking(["10h", "Jh", "Qh", "Ah", "Kh"]) ➞ "Royal Flush"<br>
> poker_hand_ranking(["3h", "5h", "Qs", "9h", "Ad"]) ➞ "High Card"<br>
> poker_hand_ranking(["10s", "10c", "8d", "10d", "10h"]) ➞ "Four of a Kind"<br>

## Pseudocode
```
For each combination (going from best to worst):
    If the player's hand satisfies the condition of that combination:
        return that combination's name
```

### Considerations:
- Every combination has it's own condition, so we can just make a unique function for each to check if it's condition is satisfied (e.g. `is_full_house(hand)`, which takes in a list of cards and outputs True/False.
- The conditions are easier to check if we pull apart the card value and suit, and if the card value is numeric rather than a string. We can do this once before very thing else by writing a function `read_card(card)` which takes in a string value for the card and outputs a `(value, suit)` pair, and applying it to each card in the hand.
- The conditions will be easier to check if the hand is sorted according to increasing card value. For example, checking for the straight-type hands becomes trivial, as does checking for pairs/triplets (since they will be next to each other).
- Aces can be either 14 or 1 in the context of a straight. I'm going to just conveniently ignore this caveat :).

# New pseudocode
```
Convert the cards in a hand to a list of (value, suit) pairs
Sort the cards in a hand by their value
For each combination (going from best to worst):
    If the player's hand satisfies the condition of that combination:
        return that combination's name
```

In [3]:
# Implement a function to convert a string-formatted card (e.g. "10h", "Jh", etc.) to a (value, suit) pair

from collections import namedtuple

Card = namedtuple('Card', ['value', 'suit'])

royal_values = {'J': 11, 'Q': 12, 'K': 13, 'A': 14}

def read_card(card):
    value, suit = card[:-1], card[-1]
    if str.isnumeric(value):
        value = int(value)
    else:
        value = royal_values[value]
    return Card(value=value, suit=suit)


# Test the function with a couple card values
assert read_card('10h') == (10, 'h')
assert read_card('2c') == (2, 'c')
assert read_card('Ad') == (14, 'd')

In [4]:
# Implement the functions to check if a hand satisfies the combination.
# Some of these will reuse others (e.g. to check if a hand is a Full House, we have to check if there is a Pair),
# so let's define them in an order that makes reuse easy.
# *Note*: Functions assume hand is sorted by card value in increasing order

def is_pair(hand):
    for i in range(len(hand) - 1):
        card1, card2 = hand[i:i + 2]
        if card1.value == card2.value:
            return True
    return False


def is_three_of_a_kind(hand):
    for i in range(len(hand) - 2):
        card1, card2, card3 = hand[i:i + 3]
        if card1.value == card2.value == card3.value:
            return True
    return False


def is_four_of_a_kind(hand):
    for i in range(len(hand) - 3):
        card1, card2, card3, card4 = hand[i:i + 4]
        if card1.value == card2.value == card3.value == card4.value:
            return True
    return False


def is_two_pair(hand):
    return (is_pair(hand[:2]) and is_pair(hand[2:4]) or
           is_pair(hand[:2]) and is_pair(hand[3:]) or
           is_pair(hand[1:3]) and is_pair(hand[3:]))


def is_full_house(hand):
    return (is_pair(hand[:2]) and is_three_of_a_kind(hand[2:]) or
           is_three_of_a_kind(hand[:3]) and is_pair(hand[3:]))


def is_straight(hand):
    for i in range(len(hand) - 1):
        card1, card2 = hand[i], hand[i + 1]
        if card1.value != card2.value - 1:
            return False
    return True


def is_flush(hand):
    for card in hand[1:]:
        if card.suit != hand[0].suit:
            return False
    return True


def is_straight_flush(hand):
    return is_flush(hand) and is_straight(hand)


def is_royal_flush(hand):
    return is_straight_flush(hand) and hand[0].value == 10

In [5]:
# Now the poker hand ranking code, which works according to
# our pseudocode (recopied here for convenience)

# Convert the cards in a hand to a list of (value, suit) pairs
# Sort the cards in a hand by their value
# For each combination (going from best to worst):
#     If the player's hand satisfies the condition of that combination:
#         return that combination's name

def card_value_sort_key(card):
    return card.value


def poker_hand_ranking(hand):
    hand = [read_card(c) for c in hand]
    hand = sorted(hand, key=card_value_sort_key)
    
    # *Note*: It is possible to loop through functions, but I don't want to 
    # do anything fancy right now. Writing out all the if statements like
    # this is equivalent to the "for" loop in our pseudocode.
    if is_royal_flush(hand):
        return 'Royal Flush'
    if is_straight_flush(hand):
        return 'Straight Flush'
    if is_four_of_a_kind(hand):
        return 'Four of a Kind'
    if is_full_house(hand):
        return 'Full House'
    if is_flush(hand):
        return 'Flush'
    if is_straight(hand):
        return 'Straight'
    if is_three_of_a_kind(hand):
        return 'Three of a Kind'
    if is_two_pair(hand):
        return 'Two Pair'
    if is_pair(hand):
        return 'Pair'
    return 'High Card'


# Test the function with a couple hands
assert poker_hand_ranking(["10h", "Jh", "Qh", "Ah", "Kh"]) == 'Royal Flush'
assert poker_hand_ranking(["3h", "5h", "Qs", "9h", "Ad"]) == 'High Card'
assert poker_hand_ranking(["10s", "10c", "8d", "10d", "10h"]) == 'Four of a Kind'

# **Stretch Content**

### 2.  Implement a function `winner_is` that returns the winner given a dictionary with different players and their hands. For example:

#### Example

We define dictionary like
```
round_1 = {"John" = ["10h", "Jh", "Qh", "Ah", "Kh"], 
        "Peter" = ["3h", "5h", "Qs", "9h", "Ad"]
}
```

Our function returns the name of the winner:
> winner_is(round_1) -> "John"

One table can have up to 10 players.


## Pseudocode
```
Set the best score found so far to 0
Set the winner so far to nobody
For every player and their hand:
    Get the combination of the player's hand (using the poker_hand_ranking function)
    Map the combination to a numeric score so that we can do comparisons
    If the player's combination gives a higher score than the best found so far:
        Update the best score found so far to that of the player's combination
        Update the winner so far to be this player
return the winner
```

In [12]:
combination_score = {'High Card': 1, 'Pair': 2, 'Two Pair': 3, 'Three of a Kind': 4,
                    'Straight': 5, 'Flush': 6, 'Full House': 7, 'Four of a Kind': 8,
                    'Straight Flush': 9, 'Royal Flush': 10}


def winner_is(poker_round):
    best_score = 0
    winners = []
    for player, hand in poker_round.items():
        combination = poker_hand_ranking(hand)
        score = combination_score[combination]
        if score == best_score:
            winners.append(player)
        elif score > best_score:
            best_score = score
            winners = [player]
    return winners


# Test the function
round_1 = {"John": ["10h", "Jh", "Qh", "Ah", "Kh"], 
           "Peter": ["10h", "Jh", "Qh", "Ah", "Kh"]}
winner_is(round_1)

['John', 'Peter']

### 3. Create a generator that randomly gives 5 cards to every player given a list of player names
#### Example

> distribute_cards(["John","Peter"])  -> round_1 = {"John" = ["10h", "Jh", "Qh", "Ah", "Kh"], 
        "Peter" = ["3h", "5h", "Qs", "9h", "Ad"]
}

## Pseudocode
```
Make a list of all the cards in a deck
Shuffle the card list
For each player in the round:
    Remove the last 5 cards from the list and assign them to be the player's hand
return every player with their corresponding hands
```

In [14]:
import random


def distribute_cards(players):
    card_values = [str(i) for i in range(2, 11)] + ['J', 'Q', 'K', 'A']
    deck = [c + 'h' for c in card_values] + \
            [c + 's' for c in card_values] + \
            [c + 'd' for c in card_values] + \
            [c + 'c' for c in card_values]
    random.shuffle(deck)
    
    poker_round = {}
    for player in players:
        hand = [deck.pop() for _ in range(5)]
        poker_round[player] = hand
        
    return poker_round


# Test out the function
distribute_cards(['Eric', 'Lynxi'])

{'Eric': ['Kc', '5h', '5s', '10c', '6c'],
 'Lynxi': ['2d', 'Ac', '10d', '3s', '8s']}